In [1]:
# importing the necessary libraries and establishing SQL Connection.
import pandas as pd
import numpy as np
import time
import collections
import mysql.connector
import math
from pandas.io import sql
import  datetime as dt
import mysql.connector as sqlcon
import os
import psycopg2 as pg
import sys
import json
import seaborn as sns
import sqlalchemy
from datetime import datetime
import utils1  
import db_config as db
import ast

pd.set_option("display.max_columns", None)
cnx_swiggy = mysql.connector.connect(user='shadowfax', password='shadowfax123', database= 'swiggy', host='test.prod.db.shadowfax.in')
RMS = db.credentials['RMS']
cnx_rms = sqlcon.connect(user = RMS['user'], password = RMS['password'] , host = RMS['host'],database = RMS['database'])
hl_read = db.credentials['hl_read']
cnx_hl = sqlcon.connect(user=hl_read['user'], password=hl_read['password'], host=hl_read['host'],database=hl_read['database'])
frodo = db.credentials['frodo']
cnx_frodo = pg.connect( host = frodo['host'] , user = frodo['user'] ,password = frodo['password'] ,  database = frodo['database'])
CPS= db.credentials['CPS']
cnx_cps= sqlcon.connect( host=CPS['host'], user=CPS['user'], password=CPS['password'],  database=CPS['database'])
smaug = db.credentials['Smaug']
cnx_smaug = pg.connect( host = smaug['host'] , user = smaug['user'] ,password = smaug['password'] ,  database = smaug['database'])
print('DONE ')




DONE 


In [2]:
i_date = dt.datetime(2023,7,31)
f_date = dt.datetime(2023,8,6)
i_date1 = i_date
s_date = i_date - dt.timedelta(days=1)
e_date= f_date + dt.timedelta(days=0)
s_date = s_date.strftime('%Y-%m-%d') + ' 18:30:00'
e_date = e_date.strftime('%Y-%m-%d') + ' 18:30:00'

i_date =  i_date.date()
f_date = f_date.date()

print(i_date)
print(f_date)
print(s_date)
print(e_date)

day_in_month = 31
print('DONE ')

2023-07-31
2023-08-06
2023-07-30 18:30:00
2023-08-06 18:30:00
DONE 


In [3]:
numdays = (f_date - i_date).days
x = numdays + 4
numdays 

6

In [4]:
#getting rider ids

def get_riders(s_date,e_date):
    query='''SELECT    
   o.rider_id
-- -----------------------------------  
FROM coreengine_order o WHERE o.scheduled_time between '{s_date}' and '{e_date}' group by 1

'''.format( s_date=s_date, e_date=e_date)
    df=pd.read_sql(query,cnx_hl)

    return df
cnx_hl = sqlcon.connect(user=hl_read['user'], password=hl_read['password'], host=hl_read['host'],database=hl_read['database'])
riders=get_riders(s_date,e_date)
rider_list = tuple(riders.rider_id.unique())

def rider_data(riders):
    query = '''select
                 ri.id as rider_id,
                case  
                    when ri.rider_type = 0  then 'Rider'
                    when ri.rider_type = 1  then 'Ops'
                    when ri.rider_type = 2  then 'Helper'
                    when ri.rider_type = 3  then 'Freelancer'
                    when ri.rider_type = 4  then 'RIDES'
                    when ri.rider_type = 5  then 'DC Staff'
                    when ri.rider_type = 6  then 'Wish'
                    when ri.rider_type = 7  then 'HULK'
                    when ri.rider_type = 8  then 'LCV'
                    when ri.rider_type = 9  then 'MILK_RUN'
                    when ri.rider_type = 10 then 'RIDER_X'
                    when ri.rider_type = 11 then 'SELLER'
                    when ri.rider_type = 12 then 'Vendor Ecom'
                    when ri.rider_type = 13 then 'PARENT_KIOSK'
                    when ri.rider_type = 14 then 'SELF_KIOSK'
                    when ri.rider_type = 15 then 'kiosk rider'
                    when ri.rider_type = 16 then 'Rider' else null end as rider_group,

                    case when rrp.vehicle_type = 3 then "cyclist" else "biker" end as vehicle_type
            from
                RMS_riderinfo ri
                left join RMS_riderpersonalinfo rrp on rrp.fk_rider_id = ri.id
            where ri.id in {riders} group by 1
            '''.format(riders = riders)
    df = sql.read_sql(query, cnx_rms)
    df['rider_id']=pd.to_numeric(df['rider_id'])
    return df

kiosk_riders = rider_data(rider_list)
print('kiosk data extracted')

kiosk_rider_list = tuple(kiosk_riders.loc[(kiosk_riders['rider_group'] == 'kiosk rider')| (kiosk_riders['rider_group'] == 'PARENT_KIOSK')].rider_id.unique())
biker = tuple(kiosk_riders.loc[kiosk_riders['vehicle_type'] == 'biker'].rider_id.unique())
cyclist = tuple(kiosk_riders.loc[kiosk_riders['vehicle_type'] == 'cyclist'].rider_id.unique())
print('DONE')

kiosk data extracted
DONE


In [5]:
# Computing PnL 


cnx_swiggy = mysql.connector.connect(user='shadowfax', password='shadowfax123', database= 'swiggy', host='test.prod.db.shadowfax.in')
RMS = db.credentials['RMS']
cnx_rms = sqlcon.connect(user = RMS['user'], password = RMS['password'] , host = RMS['host'],database = RMS['database'])
hl_read = db.credentials['hl_read']
cnx_hl = sqlcon.connect(user=hl_read['user'], password=hl_read['password'], host=hl_read['host'],database=hl_read['database'])
frodo = db.credentials['frodo']
cnx_frodo = pg.connect( host = frodo['host'] , user = frodo['user'] ,password = frodo['password'] ,  database = frodo['database'])
CPS= db.credentials['CPS']
cnx_cps= sqlcon.connect( host=CPS['host'], user=CPS['user'], password=CPS['password'],  database=CPS['database'])
smaug = db.credentials['Smaug']
cnx_smaug = pg.connect( host = smaug['host'] , user = smaug['user'] ,password = smaug['password'] ,  database = smaug['database'])


# FUNCTION DEFINING THE RPO RATES


def get_revenue_tickets_pay(s_date,e_date):
    query='''

SELECT 
o.id,
o.coid,
o.rider_id,
o.creation_time + interval 330 minute as creation_time,
date(o.creation_time + interval 330 minute) as date,
ss.chain_id,
ch.chain_name,
case when ac.name in ('Mumbai', 'Navi Mumbai','Narsinghpur') then 'Mumbai' 
 when ac.name in ('Ashok Nagar', 'Chennai') then 'Chennai' 
 when ac.name in ('Udupi', 'Manipal') then 'Manipal'  
 when ac.name in ('Howrah', 'Kolkata') then 'Kolkata' 
 when ac.name in ('Suryapet', 'Trichy') then 'Trichy' 
 when ac.name in ('Muzaffarnagar', 'Dehradun') then 'Dehradun' 
 when ac.name in ('Ambur', 'Vizianagaram') then 'Vizianagaram' 
 when ac.name in ('Bhavnagar', 'Chandigarh') then 'Chandigarh' 
 when ac.name in ('Satna', 'Bhimavaram') then 'Bhimavaram' 
 when ac.name in ('Madhyamgram', 'Siliguri') then 'Siliguri' 
 when ac.name in ('Valsad', 'Agra') then 'Agra' 
   else ac.name end as name,
o.cluster_id,
cl.cluster_name,
EXTRACT(HOUR FROM(o.creation_time+ interval 330 minute)) as hour,
case when ac.name = '' then ''
when ac.name in ('Delhi','Faridabad','Gurgaon','Ghaziabad','Noida') then 'NCR'
when ac.name = 'Bengaluru' then 'BLR'
when ac.name = 'Chennai' then 'CHN'
when ac.name = 'Pune' then 'PNQ'
when ac.name = 'Kolkata' then 'CCU'
when ac.name = 'Hyderabad' then 'HYD'
when ac.name in ('Mumbai','Navi Mumbai') then 'BOM'
else 'REST' end as city_tag,  
ov.pickup_km as FM,
p.distance as LM,
fm_normalized_order_count,
lm_normalized_order_count,
rts_normalized_order_count,
o.status,
o.payment_mode,
o.source

FROM
  coreengine_order o
  LEFT JOIN coreengine_sfxseller ss ON ss.id = o.seller_id
  LEFT JOIN coreengine_chain ch ON ch.id = ss.chain_id
  LEFT JOIN coreengine_cluster cl ON cl.id = o.cluster_id
  left join coreengine_activecities ac on ac.id=cl.fk_city_id 
  left join payments_orderinvoicedata p on p.order_id = o.id
  left join coreengine_ordervariables ov on ov.order_id=o.id
  left join order_management_ordernormalizedcount om on om.order_id = o.id

WHERE
    o.scheduled_time between '{s_date}' and '{e_date}'
 AND o.status IN (5,6) 
ORDER BY 2'''.format(s_date=s_date,e_date=e_date)
    df=pd.read_sql(query,cnx_hl)

    return df

df=get_revenue_tickets_pay(s_date,e_date)
df_master = df.copy()
print('DONE')

DONE


In [6]:
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'])
df['week'] = df['date'].dt.isocalendar().week
df['date'] = df['date'].astype(str)

# removing duplicates
print('duplicates',len(df[df[['id']].duplicated()]))
df = df.drop_duplicates(subset=['id'], keep='first')
print('duplicates are dropped now :- ',len(df[df[['id']].duplicated()]))
df = df.fillna(0)


# Defining LM buckets

def bucket(row):
    if row['LM'] <= 1 :
        return '0-1'
    elif row['LM'] <= 2 :
        return  '1-2'
    elif row['LM'] <= 3 :
        return  '2-3'
    elif row['LM'] <= 4 :
        return  '3-4'
    elif row['LM'] <= 5 :
        return  '4-5'
    elif row['LM'] <= 6 :
        return  '5-6'
    elif row['LM'] <= 7:
        return  '6-7'
    elif row['LM'] <= 8:
        return  '7-8'
    elif row['LM'] <= 9:
        return  '8-9'
    elif row['LM'] <= 10:
        return  '9-10'
    elif row['LM'] <= 11 :
        return  '10-11'
    elif row['LM'] <= 12:
        return  '11-12'
    else :
        return '12-above'
    
df['lm_buckets'] = df.apply(bucket, axis = 1)


def slot(row):
    if row['hour'] in [4,5,6,7,8,9,10,11] :
        return 'breakfast'
    elif row['hour'] in [12,13,14] :
        return 'lunch'
    elif row['hour'] in [15]:
        if row['chain_id'] == 1943:
            return 'snacks'
        else :
            return 'lunch'
    elif row['hour'] in [16,17,18]:
        return 'snacks'
    elif row['hour'] in [19,20,21,22]:
        return 'dinner'
    elif row['hour'] in [23]:
        if row['chain_id']==94 :
            return 'late_night'
        else :
            return 'dinner'
    elif row['hour'] in [0,1,2,3]:
        return 'late_night'
    
df['tagging'] = df.apply(slot, axis = 1)

df.loc[(df['tagging'] =="dinner"), 'dinner'] = 1
df.loc[(df['tagging'] =="lunch"), 'lunch'] = 1
df.loc[(df['tagging'] =="breakfast"), 'breakfast'] = 1
df.loc[(df['tagging'] =="snacks"), 'snacks'] = 1
df.loc[(df['tagging'] =="late_night"), 'late_night'] = 1


# vehicle type 
df.loc[(df['rider_id'].isin(biker)), 'vehicle_type'] = 'biker'
df.loc[(df['rider_id'].isin(cyclist)), 'vehicle_type'] = 'cyclist'
df.loc[(df['rider_id'].isin(kiosk_rider_list)), 'kiosk_tag'] = 'kiosk'

# defining MKT vertical on the basis of chains
mkt = [94,97,1943,6453,6423,74,6433,6420,6435,6405,6333,6411,6404,4660,6317,6282,6356,6383,6386,6393,6366,6084,6365,4825,6310,6344,6256,6330,6045,6221,6156,4697,6240,6339,6278,6243,6305,6238,6281,6338,6218,6354,6226,5721,6198,6296,6366,6194,6095,6292,6189,21,4679,6118,6358,6368,6244,6351,6258,5483,5917,6197,6309,6108,5192,6270,6311,6172,6284,6343,6325]   
fkhl = [4706,6143]
grocery = [6167,6295,1867]
pharma = [3832,1912,4658]
non_mkt = [6346,112,6399,6260,5196,14,147,1117,1833,5069,6287,1598,6288,4086,5501,8,6370,6421,6400]



df['vertical'] = "others"
df.loc[(df['chain_id'].isin(mkt)),'vertical'] = "mkt" 
df.loc[(df['chain_id'].isin(fkhl)),'vertical'] = "fkhl"
df.loc[(df['chain_id'].isin(grocery)),'vertical'] = "grocery"
df.loc[(df['chain_id'].isin(pharma)),'vertical'] = "pharma"
df.loc[(df['chain_id'].isin(non_mkt)),'vertical'] = "non_mkt"
df = df.fillna(0)


print('DONE ')


duplicates 0
duplicates are dropped now :-  0
DONE 


In [7]:
df['rider_id'] = df['rider_id'].astype(str)
df['date'] = df['date'].astype(str)

In [8]:
df

,id,coid,rider_id,creation_time,date,chain_id,chain_name,name,cluster_id,cluster_name,hour,city_tag,FM,LM,fm_normalized_order_count,lm_normalized_order_count,rts_normalized_order_count,status,payment_mode,source,week,lm_buckets,tagging,dinner,lunch,breakfast,snacks,late_night,vehicle_type,kiosk_tag,vertical
0,221286804,+65001,95955,2023-08-03 22:28:30,2023-08-03,112,OM SWEETS,Gurgaon,4255,GUR_Old Gurgaon_Zone 6,22,NCR,0.403,1.5597,0.0,0.0,0.0,5,0,0,31,1-2,dinner,1.0,0.0,0.0,0.0,0.0,biker,0,non_mkt
1,221711317,-7TqNadVHCxs3dcjMkAeC,2309704,2023-08-06 14:36:37,2023-08-06,6453,Cloudify (Cookr.in),Coimbatore,4904,Coi_Peelamedu,14,REST,4.901,2.9934,0.0,0.0,0.0,5,1,14,31,2-3,lunch,0.0,1.0,0.0,0.0,0.0,biker,0,mkt
2,221196244,-FtRpezsfM0GXsC24dWU9,1574303,2023-08-03 13:21:07,2023-08-03,6453,Cloudify (Cookr.in),Trichy,5030,Tri_KK-Nagar,13,REST,1.184,4.4921,0.0,0.0,0.0,5,0,14,31,4-5,lunch,0.0,1.0,0.0,0.0,0.0,biker,0,mkt
3,221511193,-nE255igiNKtqakcFX85Y,2493826,2023-08-05 13:57:14,2023-08-05,6453,Cloudify (Cookr.in),Madurai,4988,MAD_Periyar,13,REST,1.882,5.7270,0.0,0.0,0.0,5,1,14,31,5-6,lunch,0.0,1.0,0.0,0.0,0.0,biker,0,mkt
4,221588448,-Nl1jwyPdN2Iv1y10t1Pi,2657365,2023-08-05 21:06:03,2023-08-05,6453,Cloudify (Cookr.in),Madurai,4987,MAD_North,21,REST,2.885,2.0305,0.0,0.0,0.0,5,1,14,31,2-3,dinner,1.0,0.0,0.0,0.0,0.0,biker,0,mkt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890008,221551957,zzezdfnsmnl7,279818,2023-08-05 18:46:10,2023-08-05,6344,Licious - MKT,Vijayawada,4863,Vij_Benz_Circle_and_Auto_Nagar,18,REST,0.459,1.7946,0.0,0.0,0.0,5,0,13,31,1-2,snacks,0.0,0.0,0.0,1.0,0.0,biker,0,mkt
890009,220801149,zzfd6e22fstb,3461644,2023-07-31 17:15:40,2023-07-31,6344,Licious - MKT,Chennai,4016,CHE_ECR_Kottivakkam,17,CHN,2.697,2.6750,0.0,0.0,0.0,5,0,13,31,2-3,snacks,0.0,0.0,0.0,1.0,0.0,biker,0,mkt
890010,221671488,zzszq6lj7ee9,3053314,2023-08-06 11:49:42,2023-08-06,6344,Licious - MKT,Hyderabad,4145,HYD_Lingampally_Nalagandla_Beeramguda,11,HYD,2.16,4.4905,0.0,0.0,0.0,5,0,13,31,4-5,breakfast,0.0,0.0,1.0,0.0,0.0,biker,0,mkt
890011,221685298,_2Zbzir6V9wZtiJVI4q5i,3565032,2023-08-06 12:46:19,2023-08-06,6453,Cloudify (Cookr.in),Madurai,4989,MAD_Thiru Nagar,12,REST,0.987,2.0358,0.0,0.0,0.0,5,0,14,31,2-3,lunch,0.0,1.0,0.0,0.0,0.0,biker,0,mkt


In [9]:
# def payout_structures(i_date,f_date):
#     query = '''
    
# SELECT

# rpay.rider_id,
# date_start,
# date_end,
# structure_name

# from 
# payout_engine_riderpayoutstructures as rpay 
# left join payout_engine_payoutstructures as pay on rpay.payout_structure_id = pay.id

 
# where
# ((date_end >= '{i_date}' and  date_start <= '{i_date}')
# or (date_start <= '{f_date}' and date_end >= '{f_date}' ) or (date_start >= '{i_date}' and date_end <= '{f_date}' )) 

# order by 2,1
                 
#                  '''.format(i_date=i_date,f_date=f_date)
#     df = sql.read_sql(query, cnx_smaug)
#     return df



# payout_structures = payout_structures(i_date,f_date)
# payout_structures['rider_id'] = payout_structures['rider_id'].astype(str)
# print("DONE")
# ps_id = df[['rider_id','date']]
# ps_id['rider_id'] = ps_id['rider_id'].astype(str)
# ps_id['date'] = ps_id['date'].astype(str)
# ps_id = ps_id.drop_duplicates(subset=['rider_id','date'], keep='first')
# ps_id = ps_id.sort_values(by='date', ascending=True)
# ps_id = ps_id.merge(payout_structures, left_on = ['rider_id'], right_on = ['rider_id'], how= 'left')
# ps_id['date'] = pd.to_datetime(ps_id['date'], format='%Y-%m-%d')
# ps_id = ps_id.loc[(ps_id['date']>=ps_id['date_start']) & (ps_id['date']<=ps_id['date_end']) ]
# ps_id['structure_name'] = ps_id['structure_name'].astype(str)
# ps_id['rider_id'] = ps_id['rider_id'].astype(str)
# ps_id['date'] = ps_id['date'].astype(str)
# ps_id['tag']='regular'
# ps_id.loc[(ps_id['structure_name'].str.contains('super_king_premium', case = False)),'tag'  ] = 'superking'
# ps_id=ps_id.fillna(0)
# ps_id = ps_id[['rider_id','date','structure_name','tag']]
# print("duplicate entries:- ", len(ps_id[ps_id[['rider_id','date']].duplicated()]))
# print("DONE")
# ps_id.head(4)

In [10]:
# df = df.merge(ps_id, left_on = ['rider_id','date'], right_on = ['rider_id','date'], how= 'left')

In [11]:
#Analysis

last_date = dt.datetime(2023,8,6)
day_count = 7

dates = [last_date - dt.timedelta(days=x) for x in range(day_count)]
date_df=pd.DataFrame(dates,columns=['order_date'])
date_range = [str(date.strftime('%Y-%m-%d')) for date in dates] 


In [12]:
j = df.loc[(df['date'].isin(date_range))]
j = j.loc[j['vertical']=='mkt']
# j = j.loc[j['chain_id']==4825]
# j = j.loc[j['city_tag']=='CHN']
j.to_csv('W31.csv')
j

,id,coid,rider_id,creation_time,date,chain_id,chain_name,name,cluster_id,cluster_name,hour,city_tag,FM,LM,fm_normalized_order_count,lm_normalized_order_count,rts_normalized_order_count,status,payment_mode,source,week,lm_buckets,tagging,dinner,lunch,breakfast,snacks,late_night,vehicle_type,kiosk_tag,vertical
1,221711317,-7TqNadVHCxs3dcjMkAeC,2309704,2023-08-06 14:36:37,2023-08-06,6453,Cloudify (Cookr.in),Coimbatore,4904,Coi_Peelamedu,14,REST,4.901,2.9934,0.0,0.0,0.0,5,1,14,31,2-3,lunch,0.0,1.0,0.0,0.0,0.0,biker,0,mkt
2,221196244,-FtRpezsfM0GXsC24dWU9,1574303,2023-08-03 13:21:07,2023-08-03,6453,Cloudify (Cookr.in),Trichy,5030,Tri_KK-Nagar,13,REST,1.184,4.4921,0.0,0.0,0.0,5,0,14,31,4-5,lunch,0.0,1.0,0.0,0.0,0.0,biker,0,mkt
3,221511193,-nE255igiNKtqakcFX85Y,2493826,2023-08-05 13:57:14,2023-08-05,6453,Cloudify (Cookr.in),Madurai,4988,MAD_Periyar,13,REST,1.882,5.7270,0.0,0.0,0.0,5,1,14,31,5-6,lunch,0.0,1.0,0.0,0.0,0.0,biker,0,mkt
4,221588448,-Nl1jwyPdN2Iv1y10t1Pi,2657365,2023-08-05 21:06:03,2023-08-05,6453,Cloudify (Cookr.in),Madurai,4987,MAD_North,21,REST,2.885,2.0305,0.0,0.0,0.0,5,1,14,31,2-3,dinner,1.0,0.0,0.0,0.0,0.0,biker,0,mkt
5,220732809,0##38513814,2109240,2023-07-31 00:20:02,2023-07-31,6221,Dotpe-Exclusive,Pune,152,Kothrud,0,PNQ,2.425,4.2185,0.0,1.0,0.0,5,0,14,31,4-5,late_night,0.0,0.0,0.0,0.0,1.0,biker,0,mkt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890008,221551957,zzezdfnsmnl7,279818,2023-08-05 18:46:10,2023-08-05,6344,Licious - MKT,Vijayawada,4863,Vij_Benz_Circle_and_Auto_Nagar,18,REST,0.459,1.7946,0.0,0.0,0.0,5,0,13,31,1-2,snacks,0.0,0.0,0.0,1.0,0.0,biker,0,mkt
890009,220801149,zzfd6e22fstb,3461644,2023-07-31 17:15:40,2023-07-31,6344,Licious - MKT,Chennai,4016,CHE_ECR_Kottivakkam,17,CHN,2.697,2.6750,0.0,0.0,0.0,5,0,13,31,2-3,snacks,0.0,0.0,0.0,1.0,0.0,biker,0,mkt
890010,221671488,zzszq6lj7ee9,3053314,2023-08-06 11:49:42,2023-08-06,6344,Licious - MKT,Hyderabad,4145,HYD_Lingampally_Nalagandla_Beeramguda,11,HYD,2.16,4.4905,0.0,0.0,0.0,5,0,13,31,4-5,breakfast,0.0,0.0,1.0,0.0,0.0,biker,0,mkt
890011,221685298,_2Zbzir6V9wZtiJVI4q5i,3565032,2023-08-06 12:46:19,2023-08-06,6453,Cloudify (Cookr.in),Madurai,4989,MAD_Thiru Nagar,12,REST,0.987,2.0358,0.0,0.0,0.0,5,0,14,31,2-3,lunch,0.0,1.0,0.0,0.0,0.0,biker,0,mkt


In [13]:
# rr1 =j[['id','coid','rider_id','date','name','chain_id','chain_name','FM','LM','vehicle_type','lm_buckets','week','breakfast','lunch','snacks','dinner','late_night']]

# rr1['rider_id'] = rr1['rider_id'].astype(str)
# rr1


In [14]:
# import utils
# rr1 = rr1.fillna(0)
# rr1['date'] = rr1['date'].astype(str)
# rr1.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1VrjzpfVFTS4MU69SH-ZswxdL2KK7EzjxfVmOXUS6jQY/edit#gid=0'

# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'base')
# utils.update_data_in_gsheet(worksheet_, rr1, start_cell='a26397')



In [15]:
# rr = pd.read_csv('rr.csv')
# rr['rider_id'] = rr['rider_id'].astype(str)
# r1 = rr1.merge(rr, left_on = ['rider_id'], right_on = ['rider_id'], how= 'left')
# r1

In [16]:
# import utils
# r1 = r1.fillna(0)
# r1['date'] = r1['date'].astype(str)
# r1['creation_time'] = r1['creation_time'].astype(str)
# r1.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1AyMC9qL6tLG_EZghVu6f1K2nlRszi3UGbnVVUpEEVjk/edit#gid=0'


# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'base')
# utils.update_data_in_gsheet(worksheet_, r1, start_cell='a1')




In [17]:
# # Removing Duplicates
# print('duplicates',len(r1[r1[['rider_id']].duplicated()]))
# r1 = r1.sort_values(by='creation_time', ascending=True)
# r2 = r1.drop_duplicates(subset=['rider_id'], keep='first')
# print('duplicates are dropped now :- ',len(r2[r2[['rider_id']].duplicated()]))


In [18]:
# import utils
# r2 = r2.fillna(0)
# r2['date'] = r2['date'].astype(str)
# r2['creation_time'] = r2['creation_time'].astype(str)
# r2.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1AyMC9qL6tLG_EZghVu6f1K2nlRszi3UGbnVVUpEEVjk/edit#gid=0'


# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'first_order')
# utils.update_data_in_gsheet(worksheet_, r2, start_cell='a1')




In [19]:
jk = df.loc[(df['date'].isin(date_range))]
jk = jk.loc[jk['chain_id'].isin([94])]
# jk = jk.loc[jk['vertical']=='mkt']
# jk = jk.loc[jk['name']=='Mumbai']
jk.loc[jk['cluster_id'].isin([4242,4243,4244,4246,4241,461,4250]),'name'] = 'Noida1'

p  = pd.pivot_table(jk, index = ['week','name','city_tag','chain_id','chain_name','vehicle_type','date','hour'], values = ['id','breakfast','lunch','snacks','dinner','late_night','LM','rider_id'], aggfunc = {'rider_id':pd.Series.nunique,'id':pd.Series.nunique,'LM':sum,'dinner':sum,'late_night':sum,'lunch':sum,'snacks':sum,'breakfast':sum} ).rename(columns={'rider_id':'dau','id':'orders'}).reset_index()
p = p[['chain_name','chain_id','name','city_tag','date','vehicle_type','hour','orders','breakfast','lunch','snacks','dinner','late_night','LM']]
# p = p.sort_values(by='hour', ascending=True)
p = p.sort_values(by='name', ascending=True)
p = p.sort_values(by='date', ascending=True)
print('DONE ')
p



DONE 


,chain_name,chain_id,name,city_tag,date,vehicle_type,hour,orders,breakfast,lunch,snacks,dinner,late_night,LM
0,Swiggy,94,Agra,REST,2023-06-18,biker,0,5,0.0,0.0,0.0,0.0,5.0,28.800000
942,Swiggy,94,Nagpur,REST,2023-06-18,cyclist,13,1,0.0,1.0,0.0,0.0,0.0,0.400000
943,Swiggy,94,Nagpur,REST,2023-06-18,cyclist,14,1,0.0,1.0,0.0,0.0,0.0,2.400000
944,Swiggy,94,Nagpur,REST,2023-06-18,cyclist,21,1,0.0,0.0,0.0,1.0,0.0,1.500000
945,Swiggy,94,Nagpur,REST,2023-06-18,cyclist,23,1,0.0,0.0,0.0,0.0,1.0,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,Swiggy,94,Guntur,REST,2023-06-18,biker,16,13,0.0,0.0,13.0,0.0,0.0,57.900000
445,Swiggy,94,Guntur,REST,2023-06-18,biker,15,59,0.0,59.0,0.0,0.0,0.0,176.907249
444,Swiggy,94,Guntur,REST,2023-06-18,biker,14,55,0.0,55.0,0.0,0.0,0.0,172.800000
484,Swiggy,94,Gurgaon,NCR,2023-06-18,cyclist,10,16,16.0,0.0,0.0,0.0,0.0,31.200000


In [20]:
# import utils
# p = p.fillna(0)
# p['date'] = p['date'].astype(str)
# p.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1aMmohAAEf9GAQwNCgC5sB6HFce5020yJoSpPv5tELcs/edit#gid=0'


# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'base')
# utils.update_data_in_gsheet(worksheet_, p, start_cell='a44074')




True

In [ ]:
# jk = df.loc[(df['date'].isin(date_range))]
# jk = jk.loc[jk['chain_id']==1943]
# # jk = jk.loc[jk['vertical']=='mkt']
# jk = jk.loc[jk['city_tag']=='HYD']


# p  = pd.pivot_table(jk, index = ['week','name','city_tag','chain_name','vehicle_type','date','hour'], values = ['id','breakfast','lunch','snacks','dinner','late_night','LM','rider_id'], aggfunc = {'rider_id':pd.Series.nunique,'id':pd.Series.nunique,'LM':sum,'dinner':sum,'late_night':sum,'lunch':sum,'snacks':sum,'breakfast':sum,}).rename(columns={'rider_id':'dau','id':'orders'}).reset_index()
# p = p[['chain_name','week','name','city_tag','date','vehicle_type','hour','orders','breakfast','lunch','snacks','dinner','late_night','LM','dau']]
# p = p.sort_values(by='hour', ascending=True)
# p = p.sort_values(by='name', ascending=True)
# p = p.sort_values(by='date', ascending=True)
# print('DONE ')




In [ ]:
# import utils
# p = p.fillna(0)
# p['date'] = p['date'].astype(str)
# p.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1aMmohAAEf9GAQwNCgC5sB6HFce5020yJoSpPv5tELcs/edit#gid=0'


# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'zomato')
# utils.update_data_in_gsheet(worksheet_, p, start_cell='a38')




In [ ]:
# jk = df.loc[(df['date'].isin(date_range))]
# # jk = jk.loc[jk['chain_id']==1943]
# jk = jk.loc[jk['vertical']=='mkt']
# # jk = jk.loc[jk['city_tag']=='HYD']


# p  = pd.pivot_table(jk, index = ['week','name','city_tag','chain_name','vehicle_type','date','hour'], values = ['id','breakfast','lunch','snacks','dinner','late_night','LM','rider_id'], aggfunc = {'rider_id':pd.Series.nunique,'id':pd.Series.nunique,'LM':sum,'dinner':sum,'late_night':sum,'lunch':sum,'snacks':sum,'breakfast':sum,}).rename(columns={'rider_id':'dau','id':'orders'}).reset_index()
# p = p[['chain_name','week','name','city_tag','date','vehicle_type','hour','orders','breakfast','lunch','snacks','dinner','late_night','LM','dau']]
# print('DONE ')




In [ ]:
# import utils
# p = p.fillna(0)
# p['date'] = p['date'].astype(str)
# p.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1aMmohAAEf9GAQwNCgC5sB6HFce5020yJoSpPv5tELcs/edit#gid=0'


# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'mkt')
# utils.update_data_in_gsheet(worksheet_, p, start_cell='a1')




In [ ]:
# ###target sheet links 

# import utils
# link = 'https://docs.google.com/spreadsheets/d/1NsxWIf3JvuyJgHliJ5-nfZu-D44J8d3vqIEOzo20pSI/edit#gid=1230642128'
# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'lm')
# utils.update_data_in_gsheet(worksheet_, p, start_cell='a2')




In [ ]:
# #Analysis

# last_date = dt.datetime(2023,1,1)
# day_count = 2

# dates = [last_date - dt.timedelta(days=x) for x in range(day_count)]
# date_df=pd.DataFrame(dates,columns=['order_date'])
# date_range = [str(date.strftime('%Y-%m-%d')) for date in dates] 

# jk = df.loc[ (df['date'].isin(date_range)) & (df['chain_id'].isin([4825])) ]


# p  = pd.pivot_table(jk, index = ['chain_name','name','city_tag','date','hour'], values = ['id','bf','lunch','snacks','dinner','late_night','checker_mkt','checker_seller','LM'], aggfunc = {'id':pd.Series.nunique,'LM':pd.Series.mean,'dinner':sum,'late_night':sum,'lunch':sum,'snacks':sum,'bf':sum,'checker_mkt':sum,'checker_seller':sum}).rename(columns={'id':'day_orders'}).reset_index()
# p = p[['chain_name','date','name','city_tag','day_orders','hour','bf','lunch','snacks','dinner','late_night','checker_mkt','checker_seller','LM']]
# p = p.sort_values(by='day_orders', ascending=False)
# p = p.sort_values(by='chain_name', ascending=True)
# p = p.sort_values(by='name', ascending=True)


# print('DONE ')
# p



In [ ]:
# import utils
# p = p.fillna(0)
# p.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1TnBIVI7n5hu5bvmDZBrCAugnK3URhANsjILyT2P7WGc/edit#gid=2136285335'
# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'magicpin')
# utils.update_data_in_gsheet(worksheet_, p, start_cell='b10')


# END

## Pings

In [ ]:
s_date = pd.to_datetime('2023-05-19', dayfirst=True)

frodo_issues = db.credentials['frodo_issues']
cnx_frodo_issues = pg.connect( host = frodo_issues['host'] , user = frodo_issues['user'] ,password = frodo_issues['password'] ,  database = frodo_issues['database'])

def get_data(s_date,end_date):
    query='''
    select 
        b.date date, 
        EXTRACT(HOUR FROM (b.timestamp + INTERVAL '330 min')) as hour,
        coalesce(remarks->'chosen_cluster'->'city_id', remarks->'city_id') as city_id,
        coalesce(remarks-> 'chain_id') as chain_id,
        coalesce(remarks->'chosen_cluster'->'id', remarks->'cluster_id') as cluster_id,
        -- coalesce(b.remarks->>'order_paid',b.remarks->>'is_prepaid_order') as order_type,
        count(distinct case when b.serviceability_status='t' then b.id end) True_Pings,
        count(distinct b.remarks-> 'request_id') Total_pings
        
        from order_pickupdropserviceability b
    
        where 
        date = '{s_date}'
        and b.remarks-> 'chain_id' in ('94')
        group by 1,2,3,4,5
               '''.format(s_date=s_date,end_date=end_date)
    df=pd.read_sql(query,cnx_frodo_issues)
    return df

print(s_date)
pin=pd.DataFrame()
szpings=pd.DataFrame()

for i in range(1):
    print(i)
    end_date = s_date + dt.timedelta(days=1)
    frodo_issues = db.credentials['frodo_issues']
    cnx_frodo_issues = pg.connect( host = frodo_issues['host'] , user = frodo_issues['user'] ,password = frodo_issues['password'] ,  database = frodo_issues['database'])

    pin=get_data(s_date,end_date)

    hl_read = db.credentials['hl_read']
    cnx_hl = sqlcon.connect(user=hl_read['user'], password=hl_read['password'], host=hl_read['host'],database=hl_read['database'])
    final=pin
    szpings=szpings.append(final)

    
    print(s_date)
    s_date=end_date
    print(end_date)
    

    
print('DONE')


In [ ]:
pivot = pd.pivot_table(szpings, index = ['city_id','date','hour'], values = ['true_pings','total_pings'], aggfunc = {'total_pings':sum,'true_pings':sum}).reset_index()


In [ ]:
# pivot = pd.read_csv('pings.csv')
# pivot = pivot.fillna(0)
# pivot

In [ ]:
df = pd.read_csv('city_tag.csv')
pivot['city_id'] = pivot['city_id'].astype(int)
df['city_id'] = df['city_id'].astype(int)
pivot2 = pivot.merge(df, left_on = ['city_id'], right_on = ['city_id'], how= 'left')
print("Done")
pivot2 = pivot2[['city_id','date','hour','total_pings','true_pings','city_name']]
pivot2.head(5)


In [ ]:
# pivot2.loc[pivot2['city_name']=='Hyderabad']

In [ ]:
# import utils
# pivot2 = pivot2.fillna(0)
# pivot2['date'] = pivot2['date'].astype(str)
# pivot2.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1aMmohAAEf9GAQwNCgC5sB6HFce5020yJoSpPv5tELcs/edit#gid=0'


# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'pings')
# utils.update_data_in_gsheet(worksheet_, pivot2, start_cell='A1')




### ping base 2

In [ ]:
s_date = pd.to_datetime('2023-06-13', dayfirst=True)

frodo_issues = db.credentials['frodo_issues']
cnx_frodo_issues = pg.connect( host = frodo_issues['host'] , user = frodo_issues['user'] ,password = frodo_issues['password'] ,  database = frodo_issues['database'])

def get_data(s_date,end_date):
    query='''
   select 
        b.date date,
        b.id ,
        coalesce(remarks-> 'request_id') as id,
        EXTRACT(HOUR FROM (b.timestamp + INTERVAL '330 min')) as hour,
        EXTRACT(MINUTE FROM (b.timestamp + INTERVAL '330 min')) as minute,
        coalesce(remarks->'chosen_cluster'->'city_id', remarks->'city_id') as city_id,
        coalesce(remarks-> 'chain_id') as chain_id,
        coalesce(remarks->'chosen_cluster'->'id', remarks->'cluster_id') as cluster_id,
        b.pickup_lat ,
        b.pickup_long ,
        b.drop_lat ,
        b.drop_long ,
        coalesce(remarks-> 'reason') as reason,
        coalesce(remarks-> 'osrm_lm') as osrm_lm,
        b.serviceability_status,
        case when b.serviceability_status='t' then 1 end True_Pings
        
        from 
        order_pickupdropserviceability b
        where 
        date = '{s_date}'
        and b.remarks-> 'chain_id' in ('94')
               '''.format(s_date=s_date,end_date=end_date)
    df=pd.read_sql(query,cnx_frodo_issues)
    return df

print(s_date)
pin=pd.DataFrame()
szpings=pd.DataFrame()

for i in range(1):
    print(i)
    end_date = s_date + dt.timedelta(days=1)
    frodo_issues = db.credentials['frodo_issues']
    cnx_frodo_issues = pg.connect( host = frodo_issues['host'] , user = frodo_issues['user'] ,password = frodo_issues['password'] ,  database = frodo_issues['database'])

    pin=get_data(s_date,end_date)

    hl_read = db.credentials['hl_read']
    cnx_hl = sqlcon.connect(user=hl_read['user'], password=hl_read['password'], host=hl_read['host'],database=hl_read['database'])
    final=pin
    szpings=szpings.append(final)

    
    print(s_date)
    s_date=end_date
    print(end_date)
    

    
print('DONE')


In [ ]:
szpings['key1'] = szpings['hour']+szpings['chain_id']
szpings

In [ ]:
bb_df['rank_script'] = bb_df.groupby('uid')['id'].rank(method='first')

In [ ]:
szpings

In [ ]:
szpings = szpings.fillna(0)

In [ ]:
df = pd.read_csv('city_tag.csv')
szpings['city_id'] = szpings['city_id'].astype(int)
df['city_id'] = df['city_id'].astype(int)
df1 = szpings.merge(df, left_on = ['city_id'], right_on = ['city_id'], how= 'left')
print("Done")
# pivot2 = pivot2[['city_id','date','hour','total_pings','true_pings','city_name']]
df1['total_pings'] = 1
df1.head(5)

In [ ]:
pivot = pd.pivot_table(df1, index = ['reason','city_name'], values = ['true_pings','total_pings','osrm_lm'], aggfunc = {'total_pings':sum,'true_pings':sum,'osrm_lm':sum}).reset_index()
pivot


In [ ]:
# import utils
# pivot = pivot.fillna(0)
# pivot.replace([np.inf, -np.inf], 0, inplace=True)
# link = 'https://docs.google.com/spreadsheets/d/1aMmohAAEf9GAQwNCgC5sB6HFce5020yJoSpPv5tELcs/edit#gid=2003893780'


# ghseet_ = utils.open_gsheet_by_url(link)
# worksheet_ = utils.access_worksheet_by_name(ghseet_, 'jj')
# utils.update_data_in_gsheet(worksheet_, pivot, start_cell='a2')


